# Overview and Limitations

Explain the big picture of what you're exploring and why:

* Data published in 2020 shows Oregon ranked second in alcohol and drug addiction rates in the nation and last in treatment options. 

* On February 1, 2021, Oregon passed a measure that removed all criminal penalties for the possession of small doses of illegal drugs; instead of giving those caught with possession the opportunity to pay a $100 fine or to undergo a health assessment at local recovery centers. By decriminalizing personal-drug use, the heart of the bill is to say: you can’t treat addiction as a crime and disease simultaneously. 

* A year after the ballot measure was implemented, we looked at one of the state’s biggest cities Portland to gauge the impact of this new drug law. Were crime arrests really decreasing? If so, by how much? The Oregon Criminal Justice Commission predicted that arrests for low-level possession would drop by around 90%. 

* We obtained data from Portland’s Uniformed Crime Report data published by the FBI and looked at the drug and narcotic rates a year before February 2021 and around 10 months after. We then compared that to the state’s reported overall crime rates. 


Lastly, mention any major challenges you faced with respect to data. Again, you should only provide high-level detail. For example: 

> We originally hoped to analyze a 5-10 year span for all Oregon police agencies using FBI UCR crime data. Unfortunately, the UCR data lags by `X years (or months?)` and would not allow us to analyze the possible impact of Measure 110, which went into effect at the start of 2021. Therefore, we chose to focus on Portland, which provides detailed crime data on its police website and only lags by `[some amount of time]`.


## Data quality vetting

The `OffenseCategory` and `OccurDate` fields are critical to your analysis. Demonstrate some basic data quality checks for these fields:

* Show unique values for `OffenseCategory`
* Min/Max for `OccurDate`
* Any rows missing the `OffenseCategory` or `OccurDate` values?

## Key questions for analysis

Set the stage by answering some big picture questions:

* What are Offense Category counts for both years (again, this should be the total for both years)?
* Which crime categories had the most dramatic increases or decreases pre and post Measure 110 as a percentage of all crimes? *This analysis sets the stage to highlight drugs, prostitution and whatever else jumps out as noteworthy news-wise.* 
* How did crime rates differ by OffenseCategory and CrimeAgainst? 

* What percerntage of total crimes do drugs represent for 2020 and 2021?


## Key findings 

* Drug offense saw the greatest decrease in crime out of all drug offenses (by 58%) while prostitution saw the greatest increase (by 440%)
* Drug offenses made up the smallest percentage of crimes out of all offense categories during the post Measure 110 period 
* Crimes against Society saw the greatest drop out of all the three CrimeAgainst categories (People, Property, Society). Drug crimes are categorized as crimes against society. 



In [18]:
import csv, os
from datetime import datetime 
from collections import defaultdict
import pandas as pd 
import altair as alt

## Data preparation

Below we import data from the Portland Oregon `PROVIDE LINK here and brief background on the data set`. 

In [19]:
#Function that returns whether or not the date of the crime occured before or after Measure_110 was enacted 
def before_or_after(occur_date):
    enactment_date = datetime(2021, 2, 1)
    if occur_date >= enactment_date:
        return "2: post"
    elif occur_date < enactment_date: 
        return "1: pre"
    else: 
        return "ERROR"

In [20]:
data_dir = os.environ['DATA_DIR'] 
csv_file = os.path.join(data_dir, 'PortlandData.csv') 
portland_crimes = pd.read_csv(csv_file, parse_dates = ["OccurDate"])
portland_crimes = portland_crimes[portland_crimes.OccurDate >= datetime.strptime('1/1/2020', '%m/%d/%Y')].reset_index()
portland_crimes['DrugCrime'] = portland_crimes.OffenseCategory == 'Drug/Narcotic Offenses'
portland_crimes["MonthDate"] = pd.to_datetime(portland_crimes['MonthYear'], format = "%B, %Y")
portland_crimes['year'] = portland_crimes.OccurDate.dt.year
portland_crimes['month'] = portland_crimes.OccurDate.dt.month
portland_crimes['before_after'] = portland_crimes.apply(lambda row: before_or_after(row.OccurDate), axis=1)



## Data quality checks

Our analysis relies heavily on the `OffenseCategory` and `OccurDate` fields. Below we perform some basic data quality checks on these fields.

In [21]:
portland_crimes

,index,Address,CaseNumber,CrimeAgainst,Neighborhood,OccurDate,MonthYear,OccurTime,OffenseCategory,OffenseType,...,OpenDataLon,ReportDate,Report DATETIME,OffenseCount,Before or After,DrugCrime,MonthDate,year,month,before_after
0,1215,NaN,20-X5397620,Person,NaN,2020-01-01,"January, 2020",345,Assault Offenses,Simple Assault,...,NaN,1/1/2020,1/1/20,1,#REF!,False,2020-01-01,2020,1,1: pre
1,1216,NaN,20-X5397621,Person,Hazelwood,2020-01-01,"January, 2020",550,Assault Offenses,Intimidation,...,NaN,1/1/2020,1/1/20,1,#REF!,False,2020-01-01,2020,1,1: pre
2,1217,NaN,20-X5397645,Person,Collins View,2020-01-01,"January, 2020",53,Assault Offenses,Simple Assault,...,NaN,1/1/2020,1/1/20,1,#REF!,False,2020-01-01,2020,1,1: pre
3,1218,NaN,20-X5397645,Property,Collins View,2020-01-01,"January, 2020",53,Arson,Arson,...,NaN,1/1/2020,1/1/20,1,#REF!,False,2020-01-01,2020,1,1: pre
4,1219,NaN,20-X5397659,Person,Woodlawn,2020-01-01,"January, 2020",158,Assault Offenses,Simple Assault,...,NaN,1/1/2020,1/1/20,1,#REF!,False,2020-01-01,2020,1,1: pre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128555,129770,NW THURMAN ST / NW LEIF ERIKSON DR,22-903026,Property,Forest Park,2022-01-31,"January, 2022",1730,Larceny Offenses,Theft From Motor Vehicle,...,-122.725301,1/31/2022,1/31/22,1,#REF!,False,2022-01-01,2022,1,2: post
128556,129771,S WATER AVE / S CARUTHERS ST,22-29291,Property,Downtown,2022-01-31,"January, 2022",2048,Motor Vehicle Theft,Motor Vehicle Theft,...,-122.675735,1/31/2022,1/31/22,1,#REF!,False,2022-01-01,2022,1,2: post
128557,129772,SE 8TH AVE / SE TAYLOR ST,22-903034,Property,Buckman West,2022-01-31,"January, 2022",930,Larceny Offenses,Theft From Motor Vehicle,...,-122.657716,1/31/2022,1/31/22,1,#REF!,False,2022-01-01,2022,1,2: post
128558,129773,SW FAIRMOUNT BLVD / SW MARQUAM HILL RD,22-903032,Property,Southwest Hills,2022-01-31,"January, 2022",830,Larceny Offenses,Theft From Motor Vehicle,...,-122.697997,1/31/2022,1/31/22,1,#REF!,False,2022-01-01,2022,1,2: post


In [22]:
portland_crimes.OffenseCategory.unique()

array(['Assault Offenses', 'Arson', 'Vandalism',
       'Human Trafficking Offenses', 'Sex Offenses', 'Fraud Offenses',
       'Larceny Offenses', 'Weapon Law Violations', 'Motor Vehicle Theft',
       'Embezzlement', 'Robbery', 'Burglary', 'Drug/Narcotic Offenses',
       'Counterfeiting/Forgery', 'Stolen Property Offenses',
       'Extortion/Blackmail', 'Homicide Offenses',
       'Pornography/Obscene Material', 'Kidnapping/Abduction',
       'Prostitution Offenses', 'Animal Cruelty Offenses'], dtype=object)

**TODO:** As mentioned at top, some additional data quality checks to perform would include:
    
* Check for rows with malformed or missing values in the `OffenseCategory` and `OccurDate` fields.

## Data transformations

Below we apply some data transformations to facilitate downstream analysis and visualization. 

We start by creating a new `offense_category` column with less verbose names. 

> This step proved necessary to clean up chart labels in downstream data visualizations.

In [23]:
# Shorten OffenseCategory labels for cleaner charting downstream
def shorten_category(category):
    clean = category.replace(' Offenses', '')
    if clean == 'Drug/Narcotic':
        return 'Drugs'
    if clean.startswith('Weapon'):
        return 'Weapons'
    if clean == 'Human Trafficking':
        return 'Hum. Traff.'
    if clean == 'Motor Vehicle Theft':
        return 'Auto Theft'
    if clean == 'Counterfeiting/Forgery':
        return 'Counterfeiting'
    if clean == 'Extortion/Blackmail':
        return 'Extortion'
    if clean == 'Kidnapping/Abduction':
        return 'Kidnapping'
    if clean == 'Pornography/Obscene Material':
        return 'Porn/Obsc.'
    return clean
portland_crimes['offense_category'] = portland_crimes.OffenseCategory.apply(shorten_category)

Below we sanity check our new labels to see if the cleanup worked as expected.

In [24]:
portland_crimes.offense_category.unique()

array(['Assault', 'Arson', 'Vandalism', 'Hum. Traff.', 'Sex', 'Fraud',
       'Larceny', 'Weapons', 'Auto Theft', 'Embezzlement', 'Robbery',
       'Burglary', 'Drugs', 'Counterfeiting', 'Stolen Property',
       'Extortion', 'Homicide', 'Porn/Obsc.', 'Kidnapping',
       'Prostitution', 'Animal Cruelty'], dtype=object)

## Data analysis

### TODO: Crime totals

As a first step, give us the big picture totals mentioned at top under *Key questions for analysis*:

* What are crime totals for two-year period (2020 and 2021)?
* What are Offense Category counts for both years (again, this should be the total for both years, *not* the yearly breakdown)?
* Which crime categories had the most dramatic increases or decreases between 2020 and 2021 as a percentage of all crimes?



### Crimes by year and type (demo purposes)

> *Coco/Christine - Here is one fully worked out example analysis to demonstrate various techniques and the overall approach you should take for any of these analysis-oriented questions. I think you should however first work through some of the questions I mentioned at top, such as which categories saw the most dramatic increase or decrease between 2020 an 2021. That will give a better up-front picture of which categories are most newsworthy and serve to highlight the big drop in drugs. Anyhow, feel free to keep or discard the below section and data viz if it doesn't ultimately help serve your overall analysis.*

Below we attempt to analyze crimes by year and type. Specifically, we want to calculate the total count for each `OffenseCategory` for 2020 and 2021.

We remove the 2022 data because we only have partial data for the year.

> *You'd want to use the original `portland_crimes` DataFrame if you do a month-by-month analysis, since in that case it **would** make sense to retain any complete months from 2022.*

Next, we group the data by year and `Offense Category` using our new `offense_category` field.

We sum the `OffenseCount` field based on the assumption that a single incident may have multiple counts of an offense (e.g. multiple weapons charges). `This is a **HUGE** assumption that needs to be verified!!`

In [25]:
yearly_df = portland_crimes.groupby(['year', 'offense_category']).OffenseCount.sum().reset_index()
monthly_df = portland_crimes.groupby(['MonthDate', 'offense_category']).OffenseCount.sum().reset_index()

pre_post_df = portland_crimes.groupby(['before_after', 'offense_category']).OffenseCount.sum().reset_index()



In [26]:
barchart = alt.Chart(pre_post_df).mark_bar().encode(
    x = alt.X('OffenseCount:Q', stack = 'normalize') ,
    y = alt.Y('offense_category:N') ,
    color = 'before_after',
)
barchart

alt.Chart(...)

In [27]:
total_crime_monthly = portland_crimes.groupby(['MonthDate', 'DrugCrime']).OffenseCount.sum().reset_index()

In [28]:

total_crime = alt.Chart(total_crime_monthly).mark_area().encode(
    x = alt.X("MonthDate:N"),
    y = alt.Y("OffenseCount:Q"),
    color = 'DrugCrime:N'
)

#####NEED HELP HERE 

total_crime_monthly

,MonthDate,DrugCrime,OffenseCount
0,2020-01-01,False,5228
1,2020-01-01,True,138
2,2020-02-01,False,4874
3,2020-02-01,True,146
4,2020-03-01,False,4551
5,2020-03-01,True,108
6,2020-04-01,False,4471
7,2020-04-01,True,139
8,2020-05-01,False,4605
9,2020-05-01,True,188


In [29]:
pre_df = portland_crimes[portland_crimes.OccurDate < datetime(2021, 1, 1)]
post_df = portland_crimes[portland_crimes.OccurDate >= datetime(2021, 1, 1)]
pre_df = pre_df.groupby(['offense_category']).OffenseCount.sum().reset_index()
post_df = post_df.groupby(['offense_category']).OffenseCount.sum().reset_index()
merged = pd.merge(pre_df, post_df, on = 'offense_category')
merged['percent_change'] =  ((merged['OffenseCount_y'] - merged['OffenseCount_x']) / merged['OffenseCount_x']) * 100
merged

,offense_category,OffenseCount_x,OffenseCount_y,percent_change
0,Animal Cruelty,26,37,42.307692
1,Arson,504,485,-3.769841
2,Assault,8861,10113,14.129331
3,Auto Theft,6555,10188,55.423341
4,Burglary,5466,5471,0.091475
5,Counterfeiting,500,317,-36.600000
6,Drugs,976,407,-58.299180
7,Embezzlement,53,45,-15.094340
8,Extortion,30,48,60.000000
9,Fraud,3582,3268,-8.766052


In [30]:
percentchange = alt.Chart(merged).mark_bar().encode(
    alt.X('percent_change:Q'),
    alt.Y('offense_category:N'),
    color = alt.condition(alt.datum.percent_change < 0, alt.value('green'), alt.value('red'))
)
percentchange

alt.Chart(...)

In [31]:
#Figure out crime breakdown by crime against type
count_against_df = portland_crimes.groupby(['MonthDate', 'CrimeAgainst', 'before_after']).OffenseCount.sum().reset_index()
count_against_df

,MonthDate,CrimeAgainst,before_after,OffenseCount
0,2020-01-01,Person,1: pre,826
1,2020-01-01,Property,1: pre,4324
2,2020-01-01,Society,1: pre,216
3,2020-02-01,Person,1: pre,777
4,2020-02-01,Property,1: pre,4031
...,...,...,...,...
70,2021-12-01,Property,2: post,5267
71,2021-12-01,Society,2: post,99
72,2022-01-01,Person,2: post,715
73,2022-01-01,Property,2: post,4920


In [32]:
crimeagainst_bar = alt.Chart(count_against_df).mark_bar().encode(
    x = alt.Y("OffenseCount:Q", stack = 'normalize'),
    y = alt.X("CrimeAgainst:N"),
    color = "before_after:N",
)
crimeagainst_bar

alt.Chart(...)